# BIC- PSO ANN Optimisation

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Import PySwarms as instructed by pyswarms webpage
import pyswarms as ps

%load_ext autoreload
%autoreload 2

## 1. Implement a multi-layer ANN architecture

### Data Preprocessing

#### Importing the dataset (Churn Modelling)

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [806 'France' 'Male' ... 1 1 142838.64]
 [757 'Germany' 'Male' ... 1 1 127059.04]
 [570 'France' 'Female' ... 0 1 116503.92]]


#### Encoding gender label

In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

#### Encoding geography label

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

### Building the ANN

#### Initialising the ANN

In [5]:
#ANN architecture
n_inputs = 12
n_hidden = 20
n_classes = 2

num_samples = len(y)

In [6]:
#Add weight calculation
def logits_f(p):

    W1 = p[0:240].reshape((n_inputs,n_hidden))
    b1 = p[240:260].reshape((n_hidden,))
    W2 = p[260:300].reshape((n_hidden,n_classes))
    b2 = p[300:302].reshape((n_classes,))

# Forward propagation
    z1 = X.dot(W1) + b1 
    a1 = np.tanh(z1.astype(float))    
    logits = a1.dot(W2) + b2 
    return logits         

In [7]:
def f_prop(params):

    logits = logits_f(params)

# Softmax
    scores = np.exp(logits)
    probs = scores / np.sum(scores, axis=1, keepdims=True)

    corect_logprobs = -np.log(probs[range(num_samples), y])
    loss = np.sum(corect_logprobs) / num_samples

    return loss


In [8]:
def f(x):
    n_particles = x.shape[0]
    j = [f_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

## PSO Oprimisation with pyswarms

In [9]:
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9, 'k':2, 'p':2}

dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + 3*n_hidden + n_classes
optimizer = ps.single.LocalBestPSO(n_particles=50, dimensions=dimensions, options=options)

cost, pos = optimizer.optimize(f, iters=500)

2020-11-23 11:46:49,446 - pyswarms.single.local_best - INFO - Optimize for 500 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9, 'k': 2, 'p': 2}
pyswarms.single.local_best: 100%|██████████|500/500, best_cost=0.491
2020-11-23 11:48:36,716 - pyswarms.single.local_best - INFO - Optimization finished | best cost: 0.49131496181382045, best pos: [ 2.61493545e-01  3.49720629e-02  2.79237855e-01  3.68769197e-03
  2.34901648e-01  3.59094535e-03  6.75716343e-01  4.08865091e-01
  3.97900047e-03 -1.91666789e-01  3.71754386e-01  2.07576712e-01
 -3.10559614e-01  3.32459183e-01 -3.04682384e-01 -3.64210235e-01
  1.28305522e-01 -3.82918655e-01 -2.88577154e-01  1.37628929e-01
 -4.81933733e-01  2.25887517e-01 -6.87020885e-01  3.21581862e-03
  2.45942923e-01  2.24812573e-01  2.04149858e-01 -1.39963771e-01
  2.73164070e-02 -2.90212900e-01 -3.78187298e-01 -1.66896787e-01
 -2.44026485e-01  4.40295314e-01  3.14896941e-02 -3.20593746e-01
  1.36311954e-02 -9.12794946e-01 -4.53251255e-01 -3.09634781e-01
  7.98626582e-

In [10]:
def predict(pos):

    logits = logits_function(pos)
    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [11]:
(predict(pos) == y).mean()

NameError: name 'logits_function' is not defined